# Agents in RAG
- Arxiv agent
- Wikipedia agent
- Custom agent

In [ ]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

arxiv_tool.name

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

wiki_tool.name

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

text_splitter = text_splitter.split_documents(docs)
vector_db = FAISS.from_documents(text_splitter, OllamaEmbeddings())
retriever = vector_db.as_retriever()
retriever

In [ ]:
from langchain.tools.retriever import create_retriever_tool
langsmith_retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool"
)

langsmith_retriever_tool.name

In [ ]:
tools = [
    wiki_tool,
    arxiv_tool,
    langsmith_retriever_tool
]
tools

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY','')

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)


In [ ]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull('hwchase17/openai-functions-agent')

prompt.messages

In [ ]:
# Agent...

from langchain.agents import create_openai_tools_agent
from langchain.agents import create_tool_calling_agent

agent = create_openai_tools_agent(llm, tools, prompt)



In [ ]:
# Agent executer
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,tools=tools, verbose=True)
agent_executor

In [ ]:
agent_executor.invoke({"input":"Tell me about langsmith"})